In [ ]:

---

### ✅ `notebooks/LSTM_Bitcoin.ipynb` (Python Code Version)

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load and clean
df = pd.read_csv('./outputs/processed_Bitcoin.csv', parse_dates=['Date'], index_col='Date')
df = df.dropna(subset=['Log_Return', 'Volatility_7d'])

# Prepare sequences
def create_sequences(data, target, window=30):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(target[i])
    return np.array(X), np.array(y)

features = df[['Log_Return']].values
target = df['Volatility_7d'].values.reshape(-1, 1)

scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_x.fit_transform(features)
y_scaled = scaler_y.fit_transform(target)

window_size = 30
X, y = create_sequences(X_scaled, y_scaled, window=window_size)

# Split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(loss='mse', optimizer='adam')

print("Training LSTM...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred)))
print(f"Test RMSE: {rmse:.6f}")

# Save model
model.save('outputs/lstm_volatility_model.h5')
print("✅ Model saved to outputs/lstm_volatility_model.h5")

# Plot
plt.figure(figsize=(12, 5))
plt.plot(scaler_y.inverse_transform(y_test), label='Actual Volatility')
plt.plot(scaler_y.inverse_transform(y_pred), label='Predicted Volatility (LSTM)')
plt.title('Actual vs LSTM Predicted Volatility')
plt.legend()
plt.tight_layout()
plt.show()